# Arduino Serial Communication for Real Time Behavior Tracking Demo
### Jeremy Delahanty, Aneesh Bal Apr. 2021
### pySerialTransfer written by PowerBroker2, https://github.com/PowerBroker2/pySerialTransfer
---

## Import packages

In [1]:
# Import pySerialTransfer for serial comms with Arduino
from pySerialTransfer import pySerialTransfer as txfer
# Import sys for exiting program safely
import sys
# Import random for generating random numbers for demo
from random import randint
# import time for sleep for demo
import time

## Define Behaviors
Aneesh is going to classify 4 behaviors: rearing, walking, sitting, and grooming. 

Each behavior will be encoded by an integer.
### Key
- 0 = Rearing
- 1 = Walking
- 2 = Sitting
- 3 = Grooming

### Create Behavior Dictionary for print() Statements

In [2]:
behavior_dictionary = {
    0 : "Rearing!", 1 : "Walking!",
    2 : "Sitting!", 3 : "Grooming!"
}

## Transmit Behaviors to the Arduino to Power LEDs
The next block of code must be run together, so an explanation of the code follows.
### Establish a link to the Arduino: pySerialTransfer
`pySerialTransfer` is a package that allows for rapid serial (over USB) communication from Python to any Arduino board. To start a connection, the Arduino must first be connected to a computer by a USB-B cable. The connection will be held open by a `while True` statement. The package uses a function called `SerialTransfer()` which takes the following arguments:
1. COM port of the computer
2. BAUD rate or bit-communication rate
3. A true/false debug argument
Storing the `transfer` object as a variable allows us to call different properties. It is commonly saved as `link`. The link to the Arduino is then opened by:
- `link.open()`

With the link open, we create a transfer object through `tx_obj()` which simply takes the variable we want to transmit. In this case, the behavior code. The `tx_obj` returns a `packet` of information that's sent as a series of bytes the Arduino knows how to interpet. This must be saved as a variable that we can send:
- `behavior_size = link.tx_obj(behavior)`

Next, we send the behavior code using `send()`:
- `link.send(behavior_size)`

The Arduino has been coded to interpet each behavior code and turn on a specific LED.

Finally, when you wish for the demo to end, you can hit `Ctrl+C` to exit the program.

### Generate Random Numbers
In this demo, the behavior codes are created by a random number generator called by `randint()` which generates a random number in the range specified. In our case, integers from 0 - 3.

---

## Arduino Code
The Arduino requires `C++` code but the developers have made using the board easy. Here is the code that runs the Arduino:

```C++
// Control code for Arduino management of rearing experiment LEDs
// Jeremy Delahanty, Aneesh Bal Apr. 2021
// SerialTransfer.h written by PowerBroker2
// https://github.com/PowerBroker2/SerialTransfer

// Include Serital Transfer package
#include "SerialTransfer.h"

// Rename SerialTransfer to myTransfer
SerialTransfer myTransfer;

//// PIN ASSIGNMENT: LEDs ////
// output
const int rearing_led_pin = 12; // LED control for rearing behavior
const int walking_led_pin = 8; // LED control for walking behavior
const int sitting_led_pin = 7; // LED control for sitting behavior
const int grooming_led_pin = 4; // LED control for grooming behavior

// Use an integer to receive classified behavior
// The integer must be 32bit, pySerialTransfer uses 32bit packets by default
// Create the trigger_value with a size of 1
int32_t trigger_list[1];

//// SETUP ////
void setup() {
  // -- DEFINE BITRATE -- //
  // If using pySerialTransfer on Arduino UNO, Serial cannot be monitored.
  // Serial transfer of data through USB connection at BAUD of 115200
  // Arduino UNO has Serial ONLY (other boards have Serial1, Serial2)
  Serial.begin(115200);
  // Begin serial transfer
  myTransfer.begin(Serial, true);

  // -- DEFINE PIN MODES -- //
  // input, as in input to the Arduino from some kind of sensor
  // None
  // output
  // Initialize pins as digital output signals
  pinMode(rearing_led_pin, OUTPUT);
  pinMode(walking_led_pin, OUTPUT);
  pinMode(sitting_led_pin, OUTPUT);
  pinMode(grooming_led_pin, OUTPUT);
}

//// TRIGGER FUNCTION ////
// If the serial port is available:
//    receive a signal from python
//    if a 0 is sent, rearing is happening!
//    if a 1 is sent, walking is happening!
//    if a 2 is sent, sitting is happening!
//    if a 3 is sent, grooming is happening!
void behavior_trigger() {
  if (myTransfer.available())
  {
    myTransfer.rxObj(trigger_list);
    if (trigger_list[0] == 0) {
      // Rearing!
      digitalWrite(rearing_led_pin, HIGH);
      digitalWrite(walking_led_pin, LOW);
      digitalWrite(sitting_led_pin, LOW);
      digitalWrite(grooming_led_pin, LOW);
    }
    else if (trigger_list[0] == 1) {
      // Walking!
      digitalWrite(rearing_led_pin, LOW);
      digitalWrite(walking_led_pin, HIGH);
      digitalWrite(sitting_led_pin, LOW);
      digitalWrite(grooming_led_pin, LOW);
    }
    else if (trigger_list[0] == 2) {
      // Sitting!
      digitalWrite(rearing_led_pin, LOW);
      digitalWrite(walking_led_pin, LOW);
      digitalWrite(sitting_led_pin, HIGH);
      digitalWrite(grooming_led_pin, LOW);
    }
    else {
      // Grooming!
      digitalWrite(rearing_led_pin, LOW);
      digitalWrite(walking_led_pin, LOW);
      digitalWrite(sitting_led_pin, LOW);
      digitalWrite(grooming_led_pin, HIGH);
    }
  }
}

// loop
void loop() {
  behavior_trigger();
}
```

## Python Code

In [4]:
if __name__ == '__main__':
    try:
        # Establish link to the Arduino
        # SerialTransfer() takes:
        # COM Port of UART converter
        # BAUD rate, or communication rate: 115200 is faster than 9600
        # debug for if you want debug outputs when something fails
        link = txfer.SerialTransfer('COM3', 115200, debug=True) ############# REPLACE COM PORT
        # Open the link
        link.open()

        # While expeirment is running for the number of samples:
        while True:
            # Randomly generate behaviors
            behavior = randint(0,3)
            # Print behavior
            print(behavior_dictionary[behavior])
            
            # Sleep the trial for .1 second
            time.sleep(0.1)
            # Create transmission object; size refers to packet size
            behavior_size = link.tx_obj(behavior)
            # Send packet
            link.send(behavior_size)
            # Print what was sent to the Arduino
            print("Sent:\t\t{}".format(behavior))

    except KeyboardInterrupt:
        try:
            link.close()
            print("Done")
            print("Exiting...")
            sys.exit()
        except:
            pass

    except:
        import traceback
        traceback.print_exc()

        try:
            link.close()
            print("Exiting")
            sys.exit()
        except:
            pass

Sitting!
Sent:		2
Rearing!
Sent:		0
Sitting!
Sent:		2
Walking!
Sent:		1
Walking!
Sent:		1
Sitting!
Sent:		2
Rearing!
Sent:		0
Walking!
Sent:		1
Rearing!
Sent:		0
Walking!
Sent:		1
Grooming!
Sent:		3
Rearing!
Sent:		0
Sitting!
Sent:		2
Rearing!
Sent:		0
Sitting!
Sent:		2
Rearing!
Sent:		0
Grooming!
Sent:		3
Walking!
Sent:		1
Rearing!
Sent:		0
Sitting!
Sent:		2
Sitting!
Sent:		2
Grooming!
Sent:		3
Sitting!
Sent:		2
Sitting!
Sent:		2
Sitting!
Sent:		2
Grooming!
Sent:		3
Sitting!
Sent:		2
Rearing!
Sent:		0
Rearing!
Sent:		0
Walking!
Sent:		1
Grooming!
Sent:		3
Walking!
Sent:		1
Walking!
Sent:		1
Walking!
Sent:		1
Walking!
Sent:		1
Walking!
Sent:		1
Sitting!
Sent:		2
Grooming!
Sent:		3
Grooming!
Sent:		3
Sitting!
Sent:		2
Grooming!
Sent:		3
Grooming!
Sent:		3
Grooming!
Sent:		3
Rearing!
Sent:		0
Grooming!
Sent:		3
Grooming!
Sent:		3
Grooming!
Sent:		3
Grooming!
Sent:		3
Grooming!
Sent:		3
Walking!
Sent:		1
Rearing!
Sent:		0
Sitting!
Sent:		2
Rearing!
Sent:		0
Sitting!
Sent:		2
Rearing!
Sent